In [1]:
!pip install wikipedia
!pip install wikipedia-api
!pip install transformers --quiet
!pip install accelerate --quiet
!pip install bitsandbytes --quiet
!pip install langchain --quiet


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np

from HelperMethods import *

C:\Users\danpa\AppData\Local\Temp\ipykernel_27436\1718568720.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
import importlib
import HelperMethods
import pandas as pd
import numpy as np

importlib.reload(HelperMethods)
from HelperMethods import *

#### Bilboard Hot 100 data

In [4]:
charts_df = pd.read_csv('Data//charts.csv')

In [5]:
charts_df.rename(columns={'artist': 'Artist'}, inplace=True)
charts_df

,date,rank,song,Artist,last-week,peak-rank,weeks-on-board
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18
...,...,...,...,...,...,...,...
330082,1958-08-04,96,Over And Over,Thurston Harris,NaN,96,1
330083,1958-08-04,97,I Believe In You,Robert & Johnny,NaN,97,1
330084,1958-08-04,98,Little Serenade,The Ames Brothers,NaN,98,1
330085,1958-08-04,99,I'll Get By (As Long As I Have You),Billy Williams,NaN,99,1


Make the dataset smaller, only keep Top 10 entries for each week (reduce dataset by 90%)

In [6]:
top_10_each_week = charts_df.groupby('date').apply(lambda x: x.nsmallest(10, 'rank')).reset_index(drop=True)
# To display the result or work with it further
top_10_each_week.sort_values(by=['date', 'rank'])

C:\Users\danpa\AppData\Local\Temp\ipykernel_27436\2697134398.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_10_each_week = charts_df.groupby('date').apply(lambda x: x.nsmallest(10, 'rank')).reset_index(drop=True)


,date,rank,song,Artist,last-week,peak-rank,weeks-on-board
0,1958-08-04,1,Poor Little Fool,Ricky Nelson,NaN,1,1
1,1958-08-04,2,Patricia,Perez Prado And His Orchestra,NaN,2,1
2,1958-08-04,3,Splish Splash,Bobby Darin,NaN,3,1
3,1958-08-04,4,Hard Headed Woman,Elvis Presley With The Jordanaires,NaN,4,1
4,1958-08-04,5,When,Kalin Twins,NaN,5,1
...,...,...,...,...,...,...,...
33005,2021-11-06,6,Way 2 Sexy,Drake Featuring Future & Young Thug,6.0,1,8
33006,2021-11-06,7,Shivers,Ed Sheeran,9.0,7,7
33007,2021-11-06,8,Good 4 U,Olivia Rodrigo,7.0,1,24
33008,2021-11-06,9,Need To Know,Doja Cat,11.0,9,20


In [7]:
top_10_each_week['Artist'].unique()

array(['Ricky Nelson', 'Perez Prado And His Orchestra', 'Bobby Darin',
       ..., 'Coldplay x BTS', 'Wizkid Featuring Justin Bieber & Tems',
       'Doja Cat'], dtype=object)



The artist column contains multiple artists in one string, this is not ideal for analysis, we will split the arits into multiple columns (Artist1, Artist2, Artist3, etc.)

In [8]:
separated_artists = []

for artist in top_10_each_week['Artist'].unique():
    separated_artists.append(separate_artists(artist))

for i in range(1,6):
    top_10_each_week['Artists'+str(i)] = np.nan  

In [9]:
# in the loop , 'Arist'+i as the name of the column to add this artist in
for idx,row in top_10_each_week.iterrows():
    artists = separate_artists(row['Artist'])
    for i, artist in enumerate(artists):
        top_10_each_week.at[idx, 'Artist'+str(i+1)] = artist

In [ ]:
top_10_each_week[['Artist','Artist1', 'Artist2', 'Artist3', 'Artist4', 'Artist5']] 


In [ ]:
#top_10_each_week.to_csv('Data//charts10.csv', index=False)

#### Data on best selling artists and how much they sold.

In [11]:
artist_df = pd.read_csv('Data//best_selling_artists.csv')

In [12]:
artist_df['Artist'] = artist_df['Artist'].replace({'Beyoncé': 'Beyonce', 'Jay-Z': 'Jay Z', 'Pink': 'P!nk', 'B\'z': 'Bz', 'Julio Iglesias': 'Julio Iglesias Jr.', 'The Carpenters': 'Carpenters', 'Ayumi Hamasaki': 'Hamasaki Ayumi', 'Johnny Hallyday': 'Hallyday Johnny', 'Tupac Shakur': '2Pac', 'Bob Marley': 'Bob Marley & The Wailers'})

In [13]:
artist_df = artist_df.loc[artist_df['Artist'].isin(top_10_each_week[['Artist1','Artist2','Artist3','Artist4','Artist5']].values.flatten())]

#### Get Information on Personal Life,Family of artists

In [14]:
import rdflib
from rdflib import Graph, URIRef, Literal
print(artist_df['Artist'].unique())

['The Beatles' 'Elvis Presley' 'Michael Jackson' 'Elton John' 'Queen'
 'Madonna' 'Led Zeppelin' 'Rihanna' 'Pink Floyd' 'Eminem' 'Mariah Carey'
 'Taylor Swift' 'Beyonce' 'Whitney Houston' 'Eagles' 'Celine Dion'
 'The Rolling Stones' 'Drake' 'Kanye West' 'Billy Joel' 'Justin Bieber'
 'Ed Sheeran' 'Bruno Mars' 'Bruce Springsteen' 'U2' 'Aerosmith'
 'Phil Collins' 'Barbra Streisand' 'ABBA' 'Frank Sinatra' 'Katy Perry'
 'Chris Brown' 'Jay Z' 'Metallica' 'Lady Gaga' 'Lil Wayne' 'Maroon 5'
 'Adele' 'Red Hot Chili Peppers' 'Fleetwood Mac' 'Bon Jovi' 'Rod Stewart'
 'Bee Gees' 'Nicki Minaj' 'Coldplay' 'Linkin Park' 'P!nk' 'Britney Spears'
 'Shania Twain' "Guns N' Roses" 'Backstreet Boys' 'Eric Clapton'
 'Neil Diamond' 'Prince' 'Journey' 'Paul McCartney' 'Janet Jackson'
 'Kenny Rogers' 'Santana' 'George Michael' 'Dire Straits' 'The Doors'
 'Foreigner' 'Chicago' 'Bob Dylan' 'Carpenters' 'Meat Loaf' 'Cher'
 'Def Leppard' 'Genesis' 'David Bowie' 'Stevie Wonder' 'James Taylor'
 'Tina Turner' 'Olivia N

In [78]:

def send_relatives_request(artist_uri,properties,repo):
    
    results_dict = {v: [] for k, v in properties.items()}
    # Madonna's Wikidata item
    item = pywikibot.ItemPage(repo, artist_uri)
    
    # Fetch the item's data
    item.get()

    # Iterate through the specified properties
    for prop in properties:
        if prop in item.claims:
            #print(f"{properties[prop]}:")
            for claim in item.claims[prop]:
                # The target is the family member's item
                target = claim.getTarget()
                print(target)
                
                if isinstance(target, pywikibot.ItemPage):       
                    target.get()
                    #print(f"  {target.labels['en']}")
                    results_dict[properties[prop]].append(target.labels['en'])

                        
    return results_dict      

In [87]:
### DONT RUN THIS CELL, IT TAKES A LONG TIME TO FILL celebrity_uri_dict

celebrity_uri_dict = {}

In [134]:
import pywikibot
from pywikibot import Site

def search_entity_uri_by_label(label, language='en'):
    """
    Search for a Wikidata entity by its label and return its URI.
    
    Parameters:
    - label: The label of the entity to search for.
    - language: The language of the label (default is 'en' for English).
    
    Returns:
    The URI of the found entity, or None if not found.
    """
    # Connect to Wikidata
    wikidata_site = Site("wikidata", "wikidata")
    repo = wikidata_site.data_repository()
    
    # Search for entities by label
    search_results = list(repo.search_entities(label, language))
    
    if not search_results:
        print(f"No entity found for label '{label}' in language '{language}'.")
        return None
    
    # Assuming we take the first search result
    first_result = search_results[0]
    entity_id = first_result['id']
    
    # Construct the URI
    uri = entity_id
    
    return uri

# Example usage
labels =  artist_df['Artist'].unique()
labels[4] = 'Queen(band)'

#celebrity_uri_dict = {}

for label in labels:
    print(f"Searching for entity with label '{label}'..")
    uri = search_entity_uri_by_label(label)    
    celebrity_uri_dict[label] = uri
    
    if uri:
        print(f"Found URI for '{label}': {uri}")

Searching for entity with label 'The Beatles'..
Found URI for 'The Beatles': Q1299
Searching for entity with label 'Elvis Presley'..
Found URI for 'Elvis Presley': Q303
Searching for entity with label 'Michael Jackson'..
Found URI for 'Michael Jackson': Q2831
Searching for entity with label 'Elton John'..
Found URI for 'Elton John': Q2808
Searching for entity with label 'Queen(band)'..
No entity found for label 'Queen(band)' in language 'en'.
Searching for entity with label 'Madonna'..
Found URI for 'Madonna': Q345
Searching for entity with label 'Led Zeppelin'..
Found URI for 'Led Zeppelin': Q2331
Searching for entity with label 'Rihanna'..
Found URI for 'Rihanna': Q36844
Searching for entity with label 'Pink Floyd'..
Found URI for 'Pink Floyd': Q2306
Searching for entity with label 'Eminem'..
Found URI for 'Eminem': Q5608
Searching for entity with label 'Mariah Carey'..
Found URI for 'Mariah Carey': Q41076
Searching for entity with label 'Taylor Swift'..
Found URI for 'Taylor Swift':

In [30]:
import pywikibot
import ast


site = pywikibot.Site("wikidata", "wikidata")
repo = site.data_repository()

In [70]:
## URI take long time so saved the file
with open('Data//artisturis.txt', 'r') as file:
    # Read the file content
    file_content = file.read()
    # Safely evaluate the string as a Python expression
    celebrity_uri_dict = ast.literal_eval(file_content)

In [71]:
# properties that i want to extract per Artist
properties = {
        # 'P22': 'Father',
        # 'P25': 'Mother',
        'P40': 'Children',
        'P26': 'Spouse',
        'P3373': 'Siblings',
        'P264': 'RecordLabel',

    }

In [73]:
celebrity_relatives = {}
# Iterate over each celebrity and fetch their relatives
for celebrity, celebrity_uri in celebrity_uri_dict.items():
    print(f"Fetching relatives for {celebrity} ({celebrity_uri})..")
    results = send_relatives_request(celebrity_uri,properties,repo)
    #print(results)
    #print('*'*50)
    celebrity_relatives[celebrity] = results
    

Fetching relatives for The Beatles (Q1299)..
Fetching relatives for Elvis Presley (Q303)..
Fetching relatives for Michael Jackson (Q2831)..
Fetching relatives for Elton John (Q2808)..
Fetching relatives for Queen(band) (Q15862)..
Fetching relatives for Madonna (Q345)..
Fetching relatives for Led Zeppelin (Q2331)..
Fetching relatives for Rihanna (Q36844)..
Fetching relatives for Pink Floyd (Q2306)..
Fetching relatives for Eminem (Q5608)..
Fetching relatives for Mariah Carey (Q41076)..
Fetching relatives for Taylor Swift (Q26876)..
Fetching relatives for Beyonce (Q36153)..
Fetching relatives for Whitney Houston (Q34389)..
Fetching relatives for Eagles (Q219714)..
Fetching relatives for Celine Dion (Q5105)..
Fetching relatives for The Rolling Stones (Q11036)..
Fetching relatives for Drake (Q7559)..
Fetching relatives for Kanye West (Q15935)..
Fetching relatives for Billy Joel (Q194333)..
Fetching relatives for Justin Bieber (Q34086)..
Fetching relatives for Ed Sheeran (Q47447)..
Fetching 

In [66]:
# import ast
# 
# with open('Data//celebrity_relatives_dict(incomplete)', 'r') as file:
#     # Read the file content
#     file_content = file.read()
#     # Safely evaluate the string as a Python expression
#     celebrity_relatives = ast.literal_eval(file_content)

In [58]:
#subset_dict.pop('George Michael')

'Q130311'

In [ ]:
celebrity_relatives_df = pd.DataFrame(celebrity_relatives).T
celebrity_relatives_df.reset_index(inplace=True)
celebrity_relatives_df.rename(columns={'index': 'Artist'}, inplace=True)
#celebrity_relatives_df.to_csv('Data//celebrity_relatives&label.csv', index=False)
celebrity_relatives_df

In [129]:
# dictionary that maps the platform id to the platform name


def send_amount_request(artist_uri,properties,repo):
    
    results_dict = {v: [] for k, v in properties.items()}
    # Madonna's Wikidata item
    item = pywikibot.ItemPage(repo, artist_uri)
    
    # Fetch the item's data
    item.get()

    # Iterate through the specified properties
    for prop in properties:
        if prop in item.claims:
            #print(f"{properties[prop]}:")
            for claim in item.claims[prop]:
                # The target is the family member's item
                target = claim.getTarget()
                amount = target.amount
                
                
                
                #if isinstance(target, pywikibot.ItemPage):       
                #target.get()
                #print(f"  {target.labels['en']}")
                
                if prop == 'P8687':
                    platformId_uri = claim.toJSON()['qualifiers-order'][0]
                    
                    #print('platformId_uri:',platformId_uri, 'amount:',amount)
                    
                    results_dict[properties[prop]].append([platformId_uri,amount])
                else:
                    results_dict[properties[prop]].append(amount)

                        
    return results_dict  

# finding Quantity data features
# properties that i want to extract per Artist
properties_quantity = {
        'P8687' : 'socialMediaFollowers',
        'P1971': 'numberOfChildren'
    }

celebrity_quantity = {}
# Iterate over each celebrity and fetch their relatives
for celebrity, celebrity_uri in celebrity_uri_dict.items():
    print(f"Fetching quantity properties for {celebrity} ({celebrity_uri})..")
    results = send_amount_request(celebrity_uri,properties_quantity,repo)
    #print(results)
    #print('*'*50)
    celebrity_quantity[celebrity] = results

Fetching quantity properties for The Beatles (Q1299)..
Fetching quantity properties for Elvis Presley (Q303)..
Fetching quantity properties for Michael Jackson (Q2831)..
Fetching quantity properties for Elton John (Q2808)..
Fetching quantity properties for Queen(band) (Q15862)..
Fetching quantity properties for Madonna (Q345)..
Fetching quantity properties for Led Zeppelin (Q2331)..
Fetching quantity properties for Rihanna (Q36844)..
Fetching quantity properties for Pink Floyd (Q2306)..
Fetching quantity properties for Eminem (Q5608)..
Fetching quantity properties for Mariah Carey (Q41076)..
Fetching quantity properties for Taylor Swift (Q26876)..
Fetching quantity properties for Beyonce (Q36153)..
Fetching quantity properties for Whitney Houston (Q34389)..
Fetching quantity properties for Eagles (Q219714)..
Fetching quantity properties for Celine Dion (Q5105)..
Fetching quantity properties for The Rolling Stones (Q11036)..
Fetching quantity properties for Drake (Q7559)..
Fetching quan

In [135]:
platform_dict = {
    'P2397': 'Youtube',
    'P6552': 'Twitter/X'

}

def extract_most_recent_follows(celebrity_quantity, celebrity, platform_dict):

    # Initialize variables to store the max values for P6552 and P585
    max_P6552 = None
    max_P2397 = None
    
    # Filter and find max for P6552
    filtered_P6552 = [follower for follower in celebrity_quantity[celebrity]['socialMediaFollowers'] if follower[0] == 'P6552']
    if filtered_P6552:
        max_P6552 = max(filtered_P6552, key=lambda x: x[1])
    
    # Filter and find max for P585
    filtered_P2397 = [follower for follower in celebrity_quantity[celebrity]['socialMediaFollowers'] if follower[0] == 'P2397']
    if filtered_P2397:
        max_P2397 = max(filtered_P2397, key=lambda x: x[1])

    
    # Update the celebrity_quantity dictionary to only include these maximum sublists for P6552 and P585
    celebrity_quantity[celebrity]['mostRecentSocialMediaFollowers'] = {
        platform_dict['P6552']: max_P6552,
         platform_dict['P2397']: max_P2397
    }
    

for celebrity in celebrity_quantity:
    extract_most_recent_follows(celebrity_quantity, celebrity, platform_dict)

In [136]:
celebrity_quantity_df = pd.DataFrame(celebrity_quantity).T
celebrity_quantity_df.drop(columns=['socialMediaFollowers'], inplace=True)
#celebrity_quantity_df.to_csv('Data//celebrity_quantity(children&followers).csv')

In [ ]:
celebrity_quantity_df



In [1]:
import pandas as pd

sales_df = pd.read_csv('Data//best_selling_artists.csv')
quantity_df = pd.read_csv('Data//celebrity_quantity(children&followers).csv')
relatives_label_df = pd.read_csv('Data//celebrity_relatives&label.csv')
charts10_df = pd.read_csv('Data//charts10.csv')
artist_public_image_df = pd.read_csv('Data//artist_public_image.csv')
quantity_df.rename(columns={'Unnamed: 0': 'Artist'}, inplace=True)




C:\Users\danpa\AppData\Local\Temp\ipykernel_22628\81398603.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
C:\Users\danpa\AppData\Local\Temp\ipykernel_22628\81398603.py:6: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  charts10_df = pd.read_csv('Data//charts10.csv')


In [30]:
quantity_df.loc[relatives_label_df['Artist'] == 'Queen']


# change queen(band) to queen

,Artist,numberOfChildren,mostRecentSocialMediaFollowers
4,Queen,[],"{'Twitter/X': ['P6552', Decimal('2195409')], '..."


In [26]:


quantity_df['Artist'] = quantity_df['Artist'].replace({'Queen(band)': 'Queen'})
relatives_label_df['Artist'] = relatives_label_df['Artist'].replace({'Queen(band)': 'Queen'})


sales_df['Artist'] = sales_df['Artist'].replace({'Beyoncé': 'Beyonce', 'Jay-Z': 'Jay Z', 'Pink': 'P!nk', 'B\'z': 'Bz', 'Julio Iglesias': 'Julio Iglesias Jr.', 'The Carpenters': 'Carpenters', 'Ayumi Hamasaki': 'Hamasaki Ayumi', 'Johnny Hallyday': 'Hallyday Johnny', 'Tupac Shakur': '2Pac', 'Bob Marley': 'Bob Marley & The Wailers'})

artist_public_image_df['Artist'] = artist_public_image_df['Artist'].replace({'Beyoncé': 'Beyonce', 'Jay-Z': 'Jay Z', 'Pink': 'P!nk', 'B\'z': 'Bz', 'Julio Iglesias': 'Julio Iglesias Jr.', 'The Carpenters': 'Carpenters', 'Ayumi Hamasaki': 'Hamasaki Ayumi', 'Johnny Hallyday': 'Hallyday Johnny', 'Tupac Shakur': '2Pac', 'Bob Marley': 'Bob Marley & The Wailers'})

,date,rank,song,Artist,last-week,peak-rank,weeks-on-board,Artists1,Artists2,Artists3,Artists4,Artists5,Artist1,Artist2,Artist3,Artist4,Artist5
9239,1976-04-17,10,Bohemian Rhapsody,Queen,12.0,10,16,NaN,NaN,NaN,NaN,NaN,Queen,NaN,NaN,NaN,NaN
9248,1976-04-24,9,Bohemian Rhapsody,Queen,10.0,9,17,NaN,NaN,NaN,NaN,NaN,Queen,NaN,NaN,NaN,NaN
9258,1976-05-01,9,Bohemian Rhapsody,Queen,9.0,9,18,NaN,NaN,NaN,NaN,NaN,Queen,NaN,NaN,NaN,NaN
10149,1978-01-14,10,We Are The Champions,Queen,13.0,10,13,NaN,NaN,NaN,NaN,NaN,Queen,NaN,NaN,NaN,NaN
10157,1978-01-21,8,We Are The Champions,Queen,10.0,8,14,NaN,NaN,NaN,NaN,NaN,Queen,NaN,NaN,NaN,NaN
10165,1978-01-28,6,We Are The Champions,Queen,8.0,6,15,NaN,NaN,NaN,NaN,NaN,Queen,NaN,NaN,NaN,NaN
10173,1978-02-04,4,We Are The Champions,Queen,6.0,4,16,NaN,NaN,NaN,NaN,NaN,Queen,NaN,NaN,NaN,NaN
10183,1978-02-11,4,We Are The Champions,Queen,4.0,4,17,NaN,NaN,NaN,NaN,NaN,Queen,NaN,NaN,NaN,NaN
10193,1978-02-18,4,We Are The Champions,Queen,4.0,4,18,NaN,NaN,NaN,NaN,NaN,Queen,NaN,NaN,NaN,NaN
10206,1978-02-25,7,We Are The Champions,Queen,4.0,4,19,NaN,NaN,NaN,NaN,NaN,Queen,NaN,NaN,NaN,NaN


In [31]:
sales_df.to_csv('Data//best_selling_artists.csv', index=False)
artist_public_image_df.to_csv('Data//artist_public_image.csv', index=False)
relatives_label_df.to_csv('Data//celebrity_relatives&label.csv', index=False)
quantity_df.to_csv('Data//celebrity_quantity(children&followers).csv', index=False)